In [ ]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
#Authentication
client_credentials_manager = SpotifyClientCredentials(client_id='0c78fdc53b5d482992b86a7f00fc434f', client_secret='924ef3f4065d47ffbd22dfe4939c8625')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
top_songs_global_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
top_songs_global_URI = playlist_link.split("/")[-1].split("?")[0]

def get_playlist_tracks(playlist_URI):
    results = sp.playlist_tracks(playlist_URI)
    tracks_data = []
    for track in results['items']:
        audio_features = sp.audio_features(track['track']['id'])[0]
        track_data = {
            'Track Name': track['track']['name'],
            'Artist': ', '.join([artist['name'] for artist in track['track']['artists']]),
            'Release Date': track['track']['album']['release_date'],
            'duration_ms': track['track']['duration_ms'],
            'popularity': track['track']['popularity'],
            'popularity': track['track']['explicit'],
            'danceability': audio_features['danceability'],
            'energy': audio_features['energy'],
            'key': audio_features['key'],
            'loudness': audio_features['loudness'],
            'mode': audio_features['mode'],
            'speechiness': audio_features['speechiness'],
            'acousticness': audio_features['acousticness'],
            'instrumentalness': audio_features['instrumentalness'],
            'liveness': audio_features['liveness'],
            'valence': audio_features['valence'],
            'tempo': audio_features['tempo'],
            'time_signature': audio_features['time_signature']
        }
        tracks_data.append(track_data)
    return tracks_data


data = pd.DataFrame(get_playlist_tracks(top_songs_global_URI))

In [ ]:
data

,Track Name,Artist,Release Date,duration_ms,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,La Diabla,Xavi,2023-11-30,172264,False,0.751,0.8190,11,-4.372,0,0.0409,0.25500,0.000000,0.2360,0.6970,131.842,3
1,greedy,Tate McRae,2023-09-15,131872,True,0.750,0.7330,6,-3.180,0,0.0319,0.25600,0.000000,0.1140,0.8440,111.018,1
2,"One Of The Girls (with JENNIE, Lily Rose Depp)","The Weeknd, JENNIE, Lily-Rose Depp",2023-06-23,244684,False,0.522,0.6410,8,-6.287,1,0.0318,0.05450,0.000000,0.1620,0.2020,87.510,4
3,Lovin On Me,Jack Harlow,2023-11-10,138411,True,0.943,0.5580,2,-4.911,1,0.0568,0.00260,0.000002,0.0937,0.6060,104.983,4
4,My Love Mine All Mine,Mitski,2023-09-15,137773,False,0.504,0.3080,9,-14.958,1,0.0321,0.86800,0.135000,0.1580,0.1210,113.950,4
5,"yes, and?",Ariana Grande,2024-01-12,214994,True,0.788,0.7580,1,-6.513,1,0.0505,0.19800,0.000085,0.1040,0.8230,119.008,4
6,Cruel Summer,Taylor Swift,2019-08-23,178426,False,0.552,0.7020,9,-5.707,1,0.1570,0.11700,0.000021,0.1050,0.5640,169.994,4
7,Stick Season,Noah Kahan,2022-10-14,182346,False,0.662,0.4880,9,-6.894,1,0.0682,0.78200,0.000000,0.1020,0.8170,117.913,4
8,redrum,21 Savage,2024-01-12,270697,True,0.624,0.7400,2,-8.445,1,0.0481,0.00529,0.000224,0.5000,0.2460,172.089,4
9,LUNA,"Feid, ATL Jacob",2023-12-01,196800,False,0.774,0.8600,7,-2.888,0,0.1300,0.13100,0.000000,0.1160,0.4460,100.019,4


In [ ]:
top_songs_global_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
top_songs_global_URI = playlist_link.split("/")[-1].split("?")[0]
top_songs_global = sp.playlist_tracks(top_songs_global_URI, limit=100)
sp.audio_features(top_songs_global['items'][0]['track']['id'])